In [2]:
# Eleringi forecasti ja tegeliku tarbimise võrdlus, viga ja protsent
import pandas as pd
forecast_path = "C:/Users/tarmo/OneDrive/Dokumendid/GIT Andmetarkus/portfolio/Electricity-Consumption-Forecast/scripts/output/forecast_consumption_hourly_next7_tallinn_20250918_20250924.csv"
df_forecast = pd.read_csv(
    forecast_path,
    sep=';',
    decimal=',',
    skipinitialspace=True,
    encoding='cp1257'  # Või 'latin1' kui cp1257 ei tööta',
)

import pytz
dt_forecast = pd.to_datetime(df_forecast['datetime_local'], utc=False)
if dt_forecast.dt.tz is None:
    dt_forecast = dt_forecast.dt.tz_localize('Europe/Tallinn')
dt_forecast_utc = dt_forecast.dt.tz_convert('UTC')
df_forecast['datetime_hour'] = dt_forecast_utc.dt.floor('h')

# Update the actuals_path to the correct absolute path
actuals_path = "C:/Users/tarmo/OneDrive/Dokumendid/GIT Andmetarkus/portfolio/Electricity-Consumption-Forecast/scripts/elering_actual_from_20250917.csv"
df_actual = pd.read_csv(actuals_path, sep=',', decimal=',', skipinitialspace=True, encoding='utf-8')
df_actual['datetime_hour'] = pd.to_datetime(df_actual['Ajatempel (UTC)'].astype(float), unit='s', utc=True).dt.floor('h')

df_actual['Tarbimine'] = (
    df_actual['Tarbimine']
    .astype(str)
    .str.replace('"', '')
    .str.replace(',', '.')
    .str.strip()
    )
df_actual['Tarbimine'] = pd.to_numeric(df_actual['Tarbimine'], errors='coerce')
df_forecast['consumption_hourly'] = (
    df_forecast['consumption_hourly']
    .astype(str)
    .str.replace('"', '')
    .str.replace(',', '.')
    .str.strip()
    )
df_forecast['consumption_hourly'] = pd.to_numeric(df_forecast['consumption_hourly'], errors='coerce')
if hasattr(df_actual['datetime_hour'].dt, 'tz'):
    df_actual['datetime_hour'] = df_actual['datetime_hour'].dt.tz_localize(None)
if hasattr(df_forecast['datetime_hour'].dt, 'tz'):
    df_forecast['datetime_hour'] = df_forecast['datetime_hour'].dt.tz_localize(None)
df_merge = pd.merge(df_actual, df_forecast, on='datetime_hour', suffixes=('_actual', '_forecast'))
df_merge['viga'] = df_merge['Tarbimine'] - df_merge['consumption_hourly']
df_merge['viga_protsent'] = 100 * df_merge['viga'] / df_merge['Tarbimine']
print(df_merge[['datetime_hour', 'Tarbimine', 'consumption_hourly', 'viga', 'viga_protsent']].head())
df_merge.to_csv("forecast_vs_actual_comparison_hourly.csv", index=False)
print("Võrdlustabel salvestatud: forecast_vs_actual_comparison_hourly.csv")

        datetime_hour  Tarbimine  consumption_hourly       viga  viga_protsent
0 2025-09-17 21:00:00      712.1          710.404740   1.695260       0.238065
1 2025-09-17 22:00:00      681.7          675.017802   6.682198       0.980226
2 2025-09-17 23:00:00      668.8          659.912795   8.887205       1.328829
3 2025-09-18 00:00:00      676.0          658.397113  17.602887       2.603977
4 2025-09-18 01:00:00      678.7          663.745422  14.954578       2.203415
Võrdlustabel salvestatud: forecast_vs_actual_comparison_hourly.csv
